In [ ]:
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
demanda = pd.read_csv("data/demaCome.csv", engine="pyarrow", sep=",")
demanda = demanda[demanda["datetime"].dt.year == 2024]
demanda

In [ ]:
precio_bolsa = pd.read_excel(
    "data/Precio_Bolsa_Nacional_($kwh)_2024.xlsx",
    engine="openpyxl",
    parse_dates=["Fecha"],
)
precio_bolsa = (
    precio_bolsa.set_index(["Fecha"])
    .stack()
    .reset_index()
    .rename(columns={"level_1": "Hora", 0: "Precio"})
)
precio_bolsa["datetime"] = pd.to_datetime(precio_bolsa["Fecha"]) + pd.to_timedelta(
    precio_bolsa["Hora"].astype(int), unit="h"
)
precio_bolsa = precio_bolsa[precio_bolsa["datetime"].dt.year == 2024]
precio_bolsa

In [ ]:
fig = go.Figure()

# Add first plot to the first y-axis
fig.add_trace(
    go.Scatter(
        x=demanda["datetime"],
        y=demanda["dema"],
        mode="lines",
        name="Demanda",
        line=dict(color="blue"),
        yaxis="y1",
    )
)

# Add second plot to the second y-axis
fig.add_trace(
    go.Scatter(
        x=precio_bolsa["datetime"],
        y=precio_bolsa["Precio"],
        mode="lines",
        name="Precio Bolsa",
        line=dict(color="red"),
        yaxis="y2",
    )
)

# Update layout for two y-axes
fig.update_layout(
    yaxis=dict(
        title="Demanda [kW]", titlefont=dict(color="blue"), tickfont=dict(color="blue")
    ),
    yaxis2=dict(
        title="Precio Bolsa [COP/kWh]",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        anchor="x",
        overlaying="y",
        side="right",
    ),
    xaxis=dict(title="Datetime"),
    # title="Demanda and Precio Bolsa Over Time"
)

fig.show()